In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
def read_parquet_from_folder(folder_name): 
    df = pd.DataFrame()
    list_filename = []
    
    for filename in os.listdir(folder_name):   
        list_filename.append(filename) 
        new_foldername = folder_name + '/' + filename  
        for filename1 in os.listdir(new_foldername):  
            with open(os.path.join(new_foldername, filename1), errors = 'ignore', mode='r') as f1: 
                 df1 = pd.read_parquet(f1.name) 
                 df1 = df1.dropna()
                 df1['stock_id'] = list_filename[-1].strip('stock_id=')
                 df = df.append(df1)     
    return df, list_filename  
trade_df, trade_filename = read_parquet_from_folder('/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet') 


df_train = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')
u1 = df_train['stock_id'].unique()       
v1 = df_train['time_id'].unique()
list_timeid = v1.tolist()
list_timeid.sort()
stock_timeid = np.zeros([max(u1)+1, len(list_timeid)])     # output of stock_id vs time_id matrix 

for i in range(len(df_train)):
    z1 = df_train['stock_id'][i]
    z2 = df_train['time_id'][i]
    z3 = list_timeid.index(z2)
    stock_timeid[z1][z3] = df_train['target'][i]
    

def func(trade_df, trade_filename, list_timeid):
    list_filename1 = []
    StockId_timeId = []
    list_stocks = []

    folder_name1 = os.listdir('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet')   
    for i in range(56):    
        for filename1 in folder_name1[i*2:i*2+2]:
            list_filename1.append(filename1)
            new_foldername = '/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet' + '/' + filename1  
            for filename2 in os.listdir(new_foldername):  
                with open(os.path.join(new_foldername, filename2), errors = 'ignore', mode='r') as f1: 
                     df = pd.read_parquet(f1.name) 
                     df = df.dropna()
                     df['stock_id'] = list_filename1[-1].strip('stock_id=')  
                 
    
            timeId_SecondBucket1 = np.zeros([len(stock_timeid[0])]) 
            if list_filename1[-1] in trade_filename: 
               c = trade_filename.index(list_filename1[-1])
               ids1 = trade_filename[c].strip('stock_id=') 
         
               list_stocks.append(int(ids1))
               df1 = trade_df[trade_df['stock_id'] == ids1] 
               timeid_unique1 = df1['time_id'].unique() 
               timeid_unique1 = timeid_unique1.tolist()
               secondbucket1_unique = df1['seconds_in_bucket'].unique() 
             
               timeid_unique2 = df['time_id'].unique() 
               timeid_unique2 = timeid_unique2.tolist()
               secondbucket2_unique = df['seconds_in_bucket'].unique() 
           

               for q in range(len(timeid_unique1)):
                   df11 = df1[df1['time_id'] == timeid_unique1[q]] 
                   df22 = df[df['time_id'] == timeid_unique1[q]] 
                   df11 = df11.reset_index(drop=True) 
                   df22 = df22.reset_index(drop=True) 
               
                   secondsbucket1 = df11['seconds_in_bucket'].unique()  
                   secondsbucket1 = secondsbucket1.tolist()
     
                   secondsbucket2 = df22['seconds_in_bucket'].unique()  
                   secondsbucket2 = secondsbucket2.tolist()
    
            
            
                   c1 = []
                   c2 = []
                   c3 = [] 
                   timeid_seconds1 = np.zeros([len(secondsbucket1)])
             
                   for r in range(len(df22)): 
                       timeid = df22.iloc[r]['time_id']  
                       seconds = df22.iloc[r]['seconds_in_bucket']  

                       if seconds in secondsbucket1: 
                          a1 = secondsbucket1.index(seconds) 
                          b1 = df11.iloc[a1]['size'] 
                          d1 = df11.iloc[a1]['order_count']
                          a, b, c, d, e, f, g, h = df22.iloc[r]['bid_price1'], df22.iloc[r]['ask_price1'], df22.iloc[r]['bid_price2'], df22.iloc[r]['ask_price2'], df22.iloc[r]['bid_size1'], df22.iloc[r]['ask_size1'], df22.iloc[r]['bid_size2'], df22.iloc[r]['ask_size2']     
                          c1.append((a1,b1,d1))
                          c2.append((a,b,c,d,e,f,g,h)) 
                          c3.append((timeid, seconds))
               
              
                   if ((c2[0][4] != 0) & (c2[0][5] != 0)) == 1: 
                      WAP1 = ((c2[0][0]*c2[0][5])+(c2[0][1]*c2[0][4]))/(c2[0][4]+c2[0][5])            #((a*f)+(b*e))/(e+f) 
                   elif c2[0][4] == 0:
                      x22 = np.array(c2).transpose()
                      WAP1 = ((c2[0][0]*c2[0][5])+(c2[0][1]*c2[0][4]))/(np.mean(x22[4])+c2[0][5])
                   elif c2[0][5] == 0:     
                      x22 = np.array(c2).transpose()
                      WAP1 = ((c2[0][0]*c2[0][5])+(c2[0][1]*c2[0][4]))/(c2[0][4]+np.mean(x22[5]))
                   
                   if ((c2[0][6] != 0) & (c2[0][7] != 0)) == 1: 
                      WAP2 = ((c2[0][2]*c2[0][7])+(c2[0][3]*c2[0][6]))/(c2[0][6]+c2[0][7])            #((c*h)+(d*g))/(g+h) 
                   elif c2[0][6] == 0:
                      x22 = np.array(c2).transpose()
                      WAP2 = ((c2[0][2]*c2[0][7])+(c2[0][3]*c2[0][6]))/(np.mean(x22[6])+c2[0][7])
                   elif c2[0][5] == 0:     
                      x22 = np.array(c2).transpose()
                      WAP2 = ((c2[0][2]*c2[0][7])+(c2[0][3]*c2[0][6]))/(c2[0][6]+np.mean(x22[7]))   
                    
                   if WAP1 > WAP2:
                      timeid_seconds1[c1[0][0]] = WAP1*(c1[0][2]/c1[0][1])
                   else:
                      timeid_seconds1[c1[0][0]] = WAP2*(c1[0][2]/c1[0][1])       
                       
                   for s in range(len(c1)-1):
                      # bidaskSpread1 = c2[s+1][1]/c2[s+1][0]     #b/a 
                      # bidaskSpread2 = c2[s+1][3]/c2[s+1][2]     #d/c 
                       if c1[s][1] != 0:
                          ratio_price = int(c1[s+1][1]/c1[s][1])     #b1 
                       else:
                          x11 = np.array(c1).transpose()
                          ratio_price = int(c1[s+1][1]/np.mean(x11[1]))   
                          
                       if ((c2[s+1][0] != 0) & (c2[s+1][3] != 0)) == 1:   
                          ratio_Spread = int(c2[s+1][1]*c2[s+1][2]/c2[s+1][0]*c2[s+1][3])
                       elif c2[s+1][0] == 0:
                          x22 = np.array(c2).transpose()
                          ratio_Spread = int(c2[s+1][1]*c2[s+1][2]/np.mean(x22[0])*c2[s+1][3])  
                       elif c2[s+1][3] == 0:
                          x22 = np.array(c2).transpose()
                          ration_Spread = int(c2[s+1][1]*c2[s+1][2]/c2[s+1][0]*np.mean(x22[3]))      
                   
                   
                       if ((c2[s+1][4] != 0) & (c2[s+1][5] != 0)) == 1: 
                          WAP1 = ((c2[s+1][0]*c2[s+1][5])+(c2[s+1][1]*c2[s+1][4]))/(c2[s+1][4]+c2[s+1][5])            #((a*f)+(b*e))/(e+f) 
                       elif c2[s+1][4] == 0:
                          x22 = np.array(c2).transpose()
                          WAP1 = ((c2[s+1][0]*c2[s+1][5])+(c2[s+1][1]*c2[s+1][4]))/(np.mean(x22[4])+c2[s+1][5])
                       elif c2[s+1][5] == 0:     
                          x22 = np.array(c2).transpose()
                          WAP1 = ((c2[s+1][0]*c2[s+1][5])+(c2[s+1][1]*c2[s+1][4]))/(c2[s+1][4]+np.mean(x22[5]))
                   
                   
                       if ((c2[s+1][6] != 0) & (c2[s+1][7] != 0)) == 1: 
                          WAP2 = ((c2[s+1][2]*c2[s+1][7])+(c2[s+1][3]*c2[s+1][6]))/(c2[s+1][6]+c2[s+1][7])            #((c*h)+(d*g))/(g+h) 
                       elif c2[s+1][6] == 0:
                          x22 = np.array(c2).transpose()
                          WAP2 = ((c2[s+1][2]*c2[s+1][7])+(c2[s+1][3]*c2[s+1][6]))/(np.mean(x22[6])+c2[s+1][7])
                       elif c2[s+1][5] == 0:     
                          x22 = np.array(c2).transpose()
                          WAP2 = ((c2[s+1][2]*c2[s+1][7])+(c2[s+1][3]*c2[s+1][6]))/(c2[s+1][6]+np.mean(x22[7]))   
                      
                  
                       if ((ratio_price >= 1 & ratio_Spread >= 1) or (ratio_price <= 1 & ratio_Spread <= 1)) == 1: 
                          WAP = WAP1
                       else:
                          WAP = WAP2
                       timeid_seconds1[c1[s+1][0]] = WAP*(c1[s+1][2]/c1[s+1][1]) 
          
                   timeid_seconds11 = timeid_seconds1[timeid_seconds1 != 0]     
          
          
                   timeId_SecondBucket11 = []
                   for t in range(len(timeid_seconds11)-1):
                       x = np.log(timeid_seconds11[t+1]/timeid_seconds11[t])  
                       timeId_SecondBucket11.append(x**2)
                                   
                   Var_SecondBucket = (np.sum(timeId_SecondBucket11)/len(timeid_seconds11))**0.5
                   timeId_SecondBucket1[list_timeid.index(timeid_unique1[q])] = Var_SecondBucket
               print('hey shikha') 
               StockId_timeId.append(timeId_SecondBucket1)
       
    return StockId_timeId, list_stocks                      
 
StockId_timeId, list_stocks = func(trade_df, trade_filename, list_timeid)
StockId_timeId = np.array(StockId_timeId)
print('hello')   

train_x = StockId_timeId        #stockid_timeid matrix 
     
alpha = 0.0000056   # hyperparameters
beta = 0.0000068
const1 = 0.000006

p_value = []       # so we can check on diff p_value means how much test_data lags from train data
for i in range(3, 50):
    p_value.append(i)



def ARIMA_model(StockId_timeId, list_stocks, alpha, p_value, const1, stock_timeid):
    Alpha = []
    Error = []
    
    while alpha <= 0.00003:
        error1 = []
        error2 = []
        for i in p_value:
            input_mat = np.zeros([len(StockId_timeId), len(StockId_timeId[0])-i])
            for j in range(len(StockId_timeId)): 
                for k in range(i, len(StockId_timeId[j])): 
                    if StockId_timeId[j][k] != 0: 
                       x1 = 0
                       for l in range(0, i): 
                           x1 += StockId_timeId[j][k-l]*alpha + const1
                       input_mat[j][k-i] = x1
                
            output_mat = stock_timeid[list_stocks, i:len(StockId_timeId[0])]   
            err_mat = output_mat - input_mat 
            error1.append(err_mat) 
            error2.append(np.sum(err_mat))  
           
        Error.append(error2)
        Alpha.append(alpha)    
        alpha += 0.000005
       
    pvalue_list = []
    errsum_list = []
    err_list = []
    for i in range(len(Error)): 
        s = min(Error[i]) 
        t = Error[i].index(s) 
        errsum_list.append(error2[t])     
        err_list.append(error1[t])    
        pvalue_list.append(t+p_value[0]) 
    return pvalue_list, errsum_list, err_list, Alpha



pvalue_list, errsum_list, err_list, Alpha = ARIMA_model(StockId_timeId, list_stocks, alpha, p_value, const1, stock_timeid)


inde = errsum_list.index(min(errsum_list)) # so we get minimum error index
alpha1 = Alpha[inde]     # true alpha for minimum error
const2 = 0.00001

minimum_err = err_list[inde]  # minimum error


df_pvalue = pd.DataFrame(pvalue_list)
unique_pvalue = df_pvalue[0].unique()
p_index = []
for i in range(len(unique_pvalue)):
    t = 0
    for j in range(len(pvalue_list)):
        if unique_pvalue[i] == pvalue_list[j]:
           t += 1
    p_index.append(t)       

e = p_index.index(max(p_index))
Pvalue = unique_pvalue[e]        # so we get how much test data lag from train data for minimum error



def ARIMA_final(alpha1, beta, const2, StockId_timeId, list_stocks, minimum_err, Pvalue, stock_timeid):
    Beta = []
    err1 = []
    err2 = []
    
    while beta <= 0.00002:
        g1 = np.zeros([len(StockId_timeId), len(StockId_timeId[0])-Pvalue])
        g2 = stock_timeid[list_stocks, Pvalue:len(StockId_timeId[0])] 
               
        for i in range(len(StockId_timeId)):
            for j in range(Pvalue, len(StockId_timeId[i])): 
                if StockId_timeId[i][j] != 0:
                   x3 = 0
                   for k in range(0, Pvalue):
                       x3 += StockId_timeId[i][j-k]*alpha1 + const2
                   g1[i][j-Pvalue] = x3 + beta*minimum_err[i][j-Pvalue]
            
        g3 = g2 - g1
        err1.append(g3) 
        err2.append(np.sum(g3))
                
        Beta.append(beta)     
        beta += 0.000003    
        
    Min_sumerr = min(err2)
    t1 = err2.index(Min_sumerr)
    true_beta = Beta[t1]
    Min_err = err1[t1]
    return Min_sumerr, true_beta, Min_err      
 
Min_sumerr, true_beta, Min_err = ARIMA_final(alpha1, beta, const2, StockId_timeId, list_stocks, minimum_err, Pvalue, stock_timeid)
  
df_test = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/test.csv')
test_stockid = df_test['stock_id'].unique()
print(test_stockid)
test_timeid = df_test['time_id'].unique()
  
train_y1 = stock_timeid[0]       # we have to use only shopid = 0 for test bcz we only have one shopid=0 
index2 = list_stocks.index(test_stockid)  # for shopid=0
result1 = Min_err[index2]        # min error for shopid=0
result2 = np.zeros(len(train_y1)-len(result1))  # for put intial value for same dimension
result2 = result2.tolist()

for i in range(len(result1)):
    result2.append(result1[i])
    
    
train_y = train_y1 - result2     # train_y for shopid=0
train_y = train_y.reshape(len(train_y), 1)     

inv_trainX = np.linalg.pinv(train_x)  # so we project train data into shopid=0
Proj_stockid = inv_trainX*train_y
Proj_stockid = Proj_stockid.transpose()

Final_result = []
test_ID = []
for i in range(len(df_test)):
    Final_result.append(np.abs(Proj_stockid[df_test['stock_id'][i]][df_test['time_id'][i]]))
    test_ID.append(df_test['row_id']) 

submission = pd.DataFrame({'row_id':test_ID, 'target':Final_result}) # this is the final submission
submission.to_csv('/kaggle/working/submission.csv', index=False)

    
  
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session